## Preprocess and motion correct 3D movies
Step 1 of the Caiman processing pipeline for multi-layer two-photon calcium imaging movies. Assume movie format acquired using the Scope setup (i.e. different layers arranged as mosaic on top of each other).

### Imports & Setup
The first cells import the various Python modules required by the notebook. In particular, a number of modules are imported from the Caiman package. In addition, we also setup the environment so that everything works as expected.

In [ ]:
# TODO: check for unnecessary imports
# Generic imports
# from __future__ import absolute_import, division, print_function
# from builtins import *
# from __future__ import print_function

import os, sys, glob, platform, re, math
import json
import time, datetime
from functools import partial
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform
from scipy import interpolate
from tifffile import TiffFile, imread, imsave
from IPython.display import clear_output

# Import Bokeh library
from bokeh.plotting import Figure, show
from bokeh.layouts import gridplot
from bokeh.models import Range1d, CrosshairTool, HoverTool, Legend
from bokeh.io import output_notebook, export_svgs
from bokeh.models.sources import ColumnDataSource
from bokeh import palettes

%matplotlib inline

In [ ]:
# on Linux we have to add the caiman folder to Pythonpath
if platform.system() == 'Linux':
    sys.path.append(os.path.expanduser('~/caiman'))
# environment variables for parallel processing
os.environ['MKL_NUM_THREADS']='1'
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['VECLIB_MAXIMUM_THREADS']='1'

In [ ]:
# Import CaImAn and custom functions
import caiman as cm
from caiman.motion_correction import MotionCorrect
import utils, mc_utils

In [ ]:
# This has to be in a separate cell, otherwise it wont work.
from bokeh import resources
output_notebook(resources=resources.INLINE)

### Specify experimental parameters

**Update!**

Here we specify where the files are located, how the files are called, the frame rate of the acquisition, how to crop the movies and how many files to process.
- animal_folder ... the folder where the data is located on the volume `Data`.
- day_folder ...
- area_folder ...
- max_files ... how many files to process per session (0 for all)
- n_planes ... number of planes recorded in the movies
- x_crop ... fraction of x pixels to crop (i.e. 0.5 for half or >1 to specify the exact number of pixels)

In [ ]:
animal_folder = 'M3_October_2018'
day_folder = 'M3_2018-10-02'
area_folder = 'S1'
max_sessions = 10 # how many sessions to process per area

n_planes = 3 # number of planes in movie
x_crop = 0.5 # fraction of x pixels to crop (i.e. 0.5 for half)

max_group_size = 75 # if there are more files, they will be processed in groups

n_processes = 4 # number of parallel processes (None to select automatically)

# create the complete path to the data folder
if platform.system() == 'Linux':
    data_folder = '/home/ubuntu/Data'
elif platform.system() == 'Darwin':
    data_folder = '/Users/Henry/Data/temp/Dendrites_Gwen'
data_folder = os.path.join(data_folder, animal_folder, day_folder, area_folder)
data_folder

In [ ]:
# select sessions for processing
p = re.compile('\d\d-\d\d-\d\d_Live') # regular expression that should match the folder names (ie. 01-23-45_Live)
sessions = [os.path.join(data_folder, x) for (i,x) in enumerate(sorted(os.listdir(data_folder))) if p.match(x) and i <= max_sessions]

In [ ]:
tiff_files = []
xml_files = []
for i_session in sessions:
    tiff_files.append([os.path.join(i_session, x) for x in os.listdir(i_session) if x.endswith('.tif') and not 'stacked' in x][0])
    xml_files.append([os.path.join(i_session, x) for x in os.listdir(i_session) if 'parameters.xml' in x][0])

In [ ]:
# read frame rate from parameters.xml
frame_rates = []
for ix, i_session in enumerate(sessions):
    tree = ET.parse(xml_files[ix])
    root = tree.getroot()
    for child in root:
        if child.tag == 'area0': # Note: only area 0!
            fr = child.find('Framerate_Hz')
            frame_rate = float(fr.text)
            frame_rates.append(frame_rate)
    if ix <= 10:
        print('Frame rate: %1.4f Hz (%s)' % (frame_rate, os.path.split(i_session)[1]))
if ix > 10:
    print('...')

### Setup cluster
The default backend mode for parallel processing is through the multiprocessing package. This will allow us to use all the cores in the VM. Note that the `cropTif` function has to be defined before starting the cluster, so that pool workers have the function available.

In [ ]:
# start the cluster (if a cluster already exists terminate it)
if 'dview' in locals():
    dview.terminate()
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=n_processes, single_thread=False)

### Convert TIFF files to ImageJ hyperstack files
This cell calls the `mosaicToStack` function in `utils` through the multiprocessing `starmap` method to make use of multiple cores.

In [ ]:
# convert multiple planes in mosaic to 3D stack and save as IJ hyperstack
stacked_files = dview.starmap(utils.mosaicToStack, [(x, n_planes, x_crop) for x in tiff_files])

In [ ]:
print('Processing %1.0f files:' % (len(stacked_files)))
print(*stacked_files[:10], sep='\n')
if len(stacked_files) > 10:
    print('...')
    print(*stacked_files[-5:], sep='\n')

### Join cropped TIF files
Next, we create a large joined TIF file from individual cropped files. Further processing will be done on the joined file.

In [ ]:
n_groups = math.ceil(len(stacked_files) / float(max_group_size))
files_per_group = math.ceil(len(stacked_files) / n_groups)
stacked_files_by_group = []
print('Processing files in %d groups' % (n_groups))
for i_groups in range(int(n_groups)):
    start_ix = int(i_groups * files_per_group)
    stop_ix = int((i_groups+1) * files_per_group)
    stacked_files_by_group.append(stacked_files[start_ix:stop_ix])
    
    print('Group %d (%d - %d): %d files' % (i_groups+1, start_ix, stop_ix, len(stacked_files[start_ix:stop_ix])))

In [ ]:
joined_tif_list = []
json_fname_list = []
total_frames_list = []
trial_indices_list = []
for i_group, stacked_files_group in enumerate(stacked_files_by_group):
    # load movies
    movies = cm.load_movie_chain(stacked_files_group, is3D=True, outtype=np.int16)
    
    total_frames = movies.shape[0]
    total_frames_list.append(total_frames)
    n_planes = movies.shape[1]
    dims = (movies.shape[2], movies.shape[3])
    
    frames_per_movie = dview.map(utils.getFramesTif, stacked_files_group)
    
     # trial index for each frame
    trial_indices = []
    for i_frame, frame_count in enumerate(frames_per_movie):
        trial_indices = trial_indices + [i_frame]*frame_count
    trial_indices_list.append(trial_indices)
    
    for i_plane in range(n_planes):
        # derive joined file name and save
        joined_tif = '%s_%s_Join_G%d_F%d_P%d.tif' % (day_folder, area_folder, i_group, total_frames, i_plane)
        imsave(os.path.join(data_folder, joined_tif), movies[:, i_plane, :, :])
        print('Saved joined TIF file %s' % (joined_tif))
    
    movies = None # free the memory
    
    # create a Json file with information about source files
    meta = {"joined_file": joined_tif.replace('_P%d.tif' % (i_plane), ''), 
            "source_frames": frames_per_movie, 
            "source_file": [x.replace(data_folder + os.path.sep,'') for x in stacked_files_group],
            "trial_index": trial_indices,
            "frame_rate": frame_rate,
            "z_planes": n_planes
           }
    json_fname = joined_tif.replace('_P%d.tif' % (i_plane), '.json')
    with open(os.path.join(data_folder, json_fname), 'w') as fid:
        json.dump(meta, fid)
    
    # save output file names in list
    joined_tif_list.append(joined_tif.replace('_P%d.tif' % (i_plane), ''))
    json_fname_list.append(json_fname)
    
    print('Created JSON metadata file %s' % (json_fname))

### Display average signal intensity
This step is optional and useful as sanity check how the imported data looks like.

In [ ]:
# select group (0, 1, ...)
group_ix = 0

color_map = palettes.d3['Category10'][10] # colors for different planes

# prepare data structure
trial_names = [x.replace(data_folder + os.path.sep,'')[:8] for x in stacked_files_by_group[group_ix]]
trial_names_frames = [trial_names[x] for x in trial_indices_list[group_ix]]
data = {'x': np.array(range(total_frames_list[group_ix])), 
        'trial_idx': trial_indices_list[group_ix],
        'trial_name': trial_names_frames
       }

# add average for each plane
for i_plane in range(n_planes):
    tiff_file = os.path.join(data_folder, joined_tif_list[group_ix] + '_P%d.tif' % (i_plane))
    mov = cm.load(tiff_file, outtype=np.int16)

    # plot average signal intensity per frame
    frame_avg = np.mean(np.mean(mov, axis=1), axis=1)

    fieldname = 'y%s' % (i_plane)
    data[fieldname] = frame_avg
    
data_source = ColumnDataSource(data)

# create figure and plot
p = Figure(plot_width=900, plot_height=300, title=('Frame average - Group %d' % (group_ix))) 
p.add_tools(CrosshairTool(), utils.getHover())
for i_plane in range(n_planes):
    p.line('x', 'y%s' % (i_plane), source=data_source, line_width=2, color=color_map[i_plane], legend='Plane %s' % (i_plane))

show(p)

### Motion correction

First, setup the parameters for motion correction. The following parameters influence the **quality** of the motion correction:
- niter_rig ... number of iterations for rigid registration (larger = better). Little improvement likely above 5-10.
- strides ... intervals at which patches are laid out for motion correction (smaller = better)
- overlaps ... overlap between patches

Note that smaller values for strides / overlap will improve registration but also lead to NaNs in the output image. In general, there is a trade-off between the quality of registration and the presence / number of NaNs in the output (at least if there is significant motion).

In [ ]:
# parameters for motion correction
params = {'niter_rig': 5,
          'max_shifts': (int(np.round(dims[0]/10)), int(np.round(dims[1]/10))),  # maximum allow rigid shift
          # for parallelization split the movies in  num_splits chuncks across time
          'splits_rig': 50,
          # if none all the splits are processed and the movie is saved
          'num_splits_to_process_rig': None,
          # intervals at which patches are laid out for motion correction
          'strides': (24, 24),
          # overlap between pathes (size of patch strides+overlaps)
          'overlaps': (24, 24),
          # for parallelization split the movies in  num_splits chuncks across time
          'splits_els': 50,
          # if none all the splits are processed and the movie is saved
          'num_splits_to_process_els': [28, None],
          'upsample_factor_grid': 4,  # upsample factor to avoid smearing when merging patches
          # maximum deviation allowed for patch with respect to rigid shift
          'max_deviation_rigid': 3,
          # Specifies how to deal with borders. (True, False, 'copy', 'min')
          'border_nan': False,
          # Cluster config
          'dview': dview,
          # Data folder
          'data_folder': data_folder,
         }

There are also some parameters for computing the quality metrics. These probably don't have to be changed.

In [ ]:
# parameters for computing metrics
winsize = 100
swap_dim = False
resize_fact_flow = 0.2    # downsample for computing ROF

Next, we define some functions. See the function doc strings for further information.

Now we are ready to run motion correction for the joined TIF file. If there are a lot of concatenated trials, this might take a while to complete.

The following outputs will be saved:
- result of rigid motion correction in Python mmap format and as TIF file
- result of pw-rigid motion correction in Python mmap format and as TIF file

### Correlation with template - uncorrected data
First, we just get the correlation with the template for the uncorrected data. This part is optional and can be used for example to re-assign groups.

In [ ]:
# mc_init_list = []
corr_orig_list = []

t_start = time.time()
for ix_file, i_file in enumerate(joined_tif_list):
    corr_orig_list.append([])
    for i_plane in range(n_planes):
        fname = os.path.join(data_folder, i_file + '_P%d.tif' % (i_plane))
        mc_init = mc_utils.setupMC(fname, params)

        # compute initial template by binned median filtering
        template = cm.load(fname).bin_median(window=10)

        # compute metric for orig
        tmpl, corr_orig, flows_orig, norms_orig, crispness_orig = \
        cm.motion_correction.compute_metrics_motion_correction(mc_init.fname[0], template.shape[0], template.shape[1], 
                                                           swap_dim, winsize=winsize, play_flow=False, 
                                                           resize_fact_flow=0.01)
    
        # mc_init_list.append(mc_init)
        corr_orig_list[ix_file].append(corr_orig)

        clear_output()
    
# print elapsed time
t_elapsed = time.time() - t_start
print('\nFinished pre-MC in %1.2f s (%1.2f s per frame)' % (t_elapsed, t_elapsed/(sum(total_frames_list)*n_planes)))

In [ ]:
# select group (0, 1, ...)
group_ix = 0

trial_names = [x.replace(data_folder + os.path.sep,'')[:8] for x in stacked_files_by_group[group_ix]]
trial_names_frames = [trial_names[x] for x in trial_indices_list[group_ix]]

# frames = np.array(range(len(corr_orig_list[group_ix])))
data = {'x': np.array(range(len(corr_orig_list[group_ix][0]))), 
        'trial_idx': trial_indices_list[group_ix],
        'trial_name': trial_names_frames
       }

# add average for each plane
for i_plane in range(n_planes):
    fieldname = 'y%s' % (i_plane)
    data[fieldname] = corr_orig_list[group_ix][i_plane]

data_source = ColumnDataSource(data)

p = Figure(plot_width=900, plot_height=300, title=('Correlation with template - Group %d' % (group_ix))) 
p.add_tools(CrosshairTool(), utils.getHover())
for i_plane in range(n_planes):
    p.line('x', 'y%s' % (i_plane), source=data_source, line_width=2, color=color_map[i_plane], legend='Plane %s' % (i_plane))

show(p)

### Run the motion correction
Next, we run the MC itself. This can be time consuming for large datasets.

In [ ]:
t_start = time.time()
mc_list = []
for ix_file, i_file in enumerate(joined_tif_list):
    mc_list.append([])
    for i_plane in range(n_planes):
        fname = os.path.join(data_folder, i_file + '_P%d.tif' % (i_plane))
        mc = mc_utils.runMotionCorrection(fname, params)
        mc_list[ix_file].append(mc)
        print('Finished MC for %s - Plane %d of %d' % (i_file, i_plane, n_planes))
    
clear_output()

# print elapsed time
t_elapsed = time.time() - t_start
print('\nFinished MC in %1.2f s (%1.2f s per frame)' % (t_elapsed, t_elapsed/(sum(total_frames_list)*n_planes)))

if platform.system() == 'Darwin':
    os.system('say "your program has finished"')

### Assess quality of motion correction
A number of key metrics can be calculated to assess how much motion correction improved overall motion. 
1. Correlation
Correlations of each frame with the template image (binned median) for original, rigid correction and pw-rigid correction. The mean correlation gives an overall impression of motion. The minimum correlation indicates the parts of the movie that are worst affected by motion. Larger correlations indicate less motion.
2. Crispness
Crispness provides a measure of the smoothness of the corrected average image. Intuitively, a dataset with nonregistered motion will have a blurred mean image, resulting in a lower value for the total gradient field norm. Thus, larger values indicate a crisper average image and less residual motion. Crispness is calculated from the gradient field of the mean image (`np.gradient`).
3. Residual optical flow
Optic flow algorithms attempt to match each frame to the template by estimating locally smooth displacement fields. The output is an image where each pixel described the local displacement between template and frame at this point. The smaller the local displacement, the better the registration. Here we compute the matrix norm of the optic flow matrix as summary statistic.

In [ ]:
# compute quality assessment metrics
crispness = []
norms = []
corr_mean = []
corr_min = []
metrics = []
for i_group in range(n_groups):
    crispness.append([])
    norms.append([])
    corr_mean.append([])
    corr_min.append([])
    metrics.append([])
    mc_group = mc_list[i_group]
    
    # remove Pool object (required for parallel processing)
    for ix, i_mc in enumerate(mc_group):
        i_mc.dview = None
        mc_group[ix] = i_mc
    
    metrics_list = dview.starmap(mc_utils.computeMetrics, [(x, swap_dim, winsize, resize_fact_flow) for x in mc_group])
    
    # collect results
    for mtrs in metrics_list:
        metrics[i_group].append(mtrs)
        # correlations, crispness and norms of residual optic flow as indicators of registration quality
        crispness[i_group].append(np.array([mtrs['crispness_orig'], mtrs['crispness_rig'], mtrs['crispness_els']]))
        norms[i_group].append(np.array([np.mean(mtrs['norms_orig']), np.mean(mtrs['norms_rig']), np.mean(mtrs['norms_els'])]))
        corr_mean[i_group].append(np.array([np.mean(mtrs['corr_orig']), np.mean(mtrs['corr_rig']), np.mean(mtrs['corr_els'])]))
        corr_min[i_group].append(np.array([np.min(mtrs['corr_orig']), np.min(mtrs['corr_rig']), np.min(mtrs['corr_els'])]))

clear_output()

Print different metrics for raw movie and rigid / pw-rigid corrected movies.

In [ ]:
for i_group in range(n_groups):
    for i_plane in range(n_planes):
        print('MC evaluation - Group %d - Plane %d:' % (i_group, i_plane))
        if corr_mean[i_group][i_plane][0] > corr_mean[i_group][i_plane][1] or corr_mean[i_group][i_plane][0] > corr_mean[i_group][i_plane][2]:
            print('\x1b[1;03;31m' + 'Mean corr - raw / rigid / pw_rigid: ' + str(['{:.2f}'.format(i) for i in corr_mean[i_group][i_plane]]) + '\x1b[0m')
        else:
            print('Mean corr - raw / rigid / pw_rigid: ' + str(['{:.2f}'.format(i) for i in corr_mean[i_group][i_plane]]))

        if corr_min[i_group][i_plane][0] > corr_min[i_group][i_plane][1] or corr_min[i_group][i_plane][0] > corr_min[i_group][i_plane][2]:
            print('\x1b[1;03;31m'+'Min corr - raw / rigid / pw_rigid: ' 
                  + str(['{:.2f}'.format(i) for i in corr_min[i_group][i_plane]])+ '\x1b[0m')
        else:
            print('Min corr - raw / rigid / pw_rigid: ' + str(['{:.2f}'.format(i) for i in corr_min[i_group][i_plane]]))
        if crispness[i_group][i_plane][0] > crispness[i_group][i_plane][1] or crispness[i_group][i_plane][0] > crispness[i_group][i_plane][2]:
            print('\x1b[1;03;31m'+'Crispness - raw / rigid / pw_rigid: ' 
                  + str(['{:.0f}'.format(i) for i in crispness[i_group][i_plane]]) + '\x1b[0m')
        else:
            print('Crispness - raw / rigid / pw_rigid: ' + str(['{:.0f}'.format(i) for i in crispness[i_group][i_plane]]))
        if norms[i_group][i_plane][0] < norms[i_group][i_plane][1] or norms[i_group][i_plane][0] < norms[i_group][i_plane][2]:
            print('\x1b[1;03;31m'+'Norms - raw / rigid / pw_rigid: ' 
                  + str(['{:.0f}'.format(i) for i in norms[i_group][i_plane]]) + '\x1b[0m')
        else:
            print('Norms - raw / rigid / pw_rigid: ' + str(['{:.2f}'.format(i) for i in norms[i_group][i_plane]]))

    print('\n')

#### Correlation with template image
Plot correlations of each frame with the template image (binned median) for original, rigid correction and pw-rigid correction. The bokeh plotting library provides a toolbar for interaction with the plot.

In [ ]:
# select group (0, 1, ...)
group_ix = 0
# select plane (0, 1, ..)
plane_ix = 2

p1 = Figure(plot_width=900, plot_height=300, title=('Correlation with template - Group %d - Plane %d' % (group_ix, plane_ix))) 
frames = np.array(range(len(metrics[group_ix][plane_ix]['corr_orig'])))
p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_orig']), line_width=2, legend='Original', color='blue')
p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_rig']), line_width=2, legend='Rigid', color='orange')
p1.line(frames,np.array(metrics[group_ix][plane_ix]['corr_els']), line_width=2, legend='PW-Rigid', color='green')

p2 = Figure(plot_width=250, plot_height=250)
p2.circle(np.array(metrics[group_ix][plane_ix]['corr_orig']), np.array(metrics[group_ix][plane_ix]['corr_rig']), size=5)
p2.line([0,1],[0,1], line_width=1, color='black', line_dash='dashed')
p2.xaxis.axis_label = 'Original'
p2.yaxis.axis_label = 'Rigid'

p3 = Figure(plot_width=250, plot_height=250)
p3.circle(np.array(metrics[group_ix][plane_ix]['corr_rig']), np.array(metrics[group_ix][plane_ix]['corr_els']), size=5)
p3.line([0,1],[0,1], line_width=1, color='black', line_dash='dashed')
p3.xaxis.axis_label = 'Rigid'
p3.yaxis.axis_label = 'PW-Rigid'

# make a grid
grid = gridplot([[p1, None], [p2, p3]], sizing_mode='fixed', toolbar_location='left')

show(grid)

#### Residual optic flow
Optic flow algorithms attempt to match each frame to the template by estimating locally smooth displacement fields. The output is an image where each pixel described the local displacement between template and frame at this point. The smaller the local displacement, the better the registration. Norms are the matrix norms of the optic flow matrix.

In [ ]:
# select group (0, 1, ...)
group_ix = 0
# select plane (0, 1, ..)
plane_ix = 0

# plot the results of Residual Optical Flow
metrics_files = [mc_list[group_ix][plane_ix].fname_tot_els[0][:-4] + '_metrics.npz', 
                 mc_list[group_ix][plane_ix].fname_tot_rig[0][:-4] + '_metrics.npz', 
                 mc_list[group_ix][plane_ix].fname[0][:-4] + '_metrics.npz']

plt.figure(figsize = (20,10))
for cnt, fl, metr in zip(range(len(metrics_files)),metrics_files,['pw_rigid','rigid','raw']):
    with np.load(fl) as ld:
        print('Correction: %s' % (metr))
        print('Norms: %1.2f +- %1.2f' % (np.mean(ld['norms']), np.std(ld['norms'])))
        
        plt.subplot(len(metrics_files), 3, 1 + 3 * cnt)
        plt.ylabel(metr)
                   
        if metr == 'raw':
            mean_img = np.mean(cm.load(mc.fname[0]), axis=0)
        elif metr == 'rigid':
            mean_img = np.mean(cm.load(mc.fname_tot_rig[0]), axis=0)
        elif metr == 'pw_rigid':
            mean_img = np.mean(cm.load(mc.fname_tot_els[0]), axis=0)
        
        lq, hq = np.nanpercentile(mean_img, [.5, 99.5])
        plt.imshow(mean_img, vmin=lq, vmax=hq, cmap='gray')
        if not cnt:
            plt.title('Mean')
        plt.subplot(len(metrics_files), 3, 3 * cnt + 2)
        plt.imshow(ld['img_corr'], vmin=0, vmax=.5, cmap='gray')
        if not cnt:
            plt.title('Correlation image')
        plt.subplot(len(metrics_files), 3, 3 * cnt + 3)
        flows = ld['flows']
        plt.imshow(np.mean(np.sqrt(flows[:, :, :, 0]**2 + flows[:, :, :, 1]**2), 0), vmin=0, vmax=0.5, cmap='gray')
        plt.colorbar()
        if not cnt:
            plt.title('Mean optical flow')
plt.suptitle('Residual Optic Flow - Group %d - Plane %d' % (group_ix, plane_ix), fontsize=22);

### Detect frames with bad motion
Identify frames with significant residual motion (low correlation with template). Write a JSON file with criterion and indices of frames matching the criterion. This file can be used in further analysis to exclude the frames corrupted by motion.

In [ ]:
thresh = [
    [0.2, 0.4, 0.4]
] # find frames where value is less than criterion (one value per group and plane)

for i_group in range(n_groups):
    for i_plane in range(n_planes):
        print('Group %d - Plane %d' % (i_group, i_plane))
        # pw-rigid registration
        criterion = 'corr_els'
        bad_frames = [ix for ix, i in enumerate(metrics[i_group][i_plane][criterion]) if i < thresh[i_group][i_plane]]
        print('%1.0f frames matching criterion after pw-rigid registration.' % (len(bad_frames)))
        mc_utils.writeJsonBadFrames(criterion, thresh[i_group][i_plane], bad_frames, mc_list[i_group][i_plane], 'els', data_folder)
        # rigid registration
        criterion = 'corr_rig'
        bad_frames = [ix for ix, i in enumerate(metrics[i_group][i_plane][criterion]) if i < thresh[i_group][i_plane]]
        print('\n%1.0f frames matching criterion after rigid registration.' % (len(bad_frames)))
        mc_utils.writeJsonBadFrames(criterion, thresh[i_group][i_plane], bad_frames, mc_list[i_group][i_plane], 'rig', data_folder)
        print('\n')